# <span style="color:#ac0707;">Large Language Models with Semantic Search - Projeto de finalização do curso</span>

Este projeto foi criado a partir da finalização do curso da deeplearning.ai - Large Language Models with Semantic Search

O curso, em parceria com a Cohere, tem como objetivo utilizar LLMs (Large Language Models) para busca semântica (ou busca por similaridade).


Peça chave deste curso são os ***embeddings*** cujo conceito será explorado no primeiro tópico do projeto a seguir.

# <span style="color: #6850e5;">Análise da relação entre filmes brasileiros usando Embeddings</span>

O objetivo desta análise é utilizar ***embeddings*** para verificar a relação de proximidade entre diversos filmes brasileiros.

*"***Embeddings*** são representações numéricas usadas em aprendizado de máquina e processamento de linguagem natural (NLP) para capturar semelhanças entre objetos em um espaço vetorial. Elas podem ser de palavras, documentos, imagens..."*

Para mapear nosso catálogo de filmes com ***embeddings*** utilizaremos a API da ***Cohere***. 

O mesmo poderia ser feito com as APIs da ***OpenAI*** pois apesar de serem empresas diferentes, com APIs diferentes, as técnicas empregadas aqui podem ser utilizadas com ambas.

Referência: https://cohere.com/

Esta análise seguirá as seguintes etapas:

1) Carregar e tratar os dados originais do nosso dataset
2) Extrair os embeddings apenas dos títulos originais dos filmes e visualizar como eles se relacionam
3) Extrair os embeddings dos resumos dos filmes ( *overview* ), que no dataset utilizado estão em inglês, e visualizar como os filmes se relacionam neste novo mapeamento.

***Obs***: Você pode baixar este notebook para estudo porém para conseguir rodá-lo corretamente será preciso gerar uma API Key no site da ***Cohere*** https://dashboard.cohere.com/welcome/register e substituí-la pela variável de ambiente ***COHERE_API_KEY***

## Importando as bibliotecas necessárias

In [511]:
%pip install -q cohere==4.20.0 umap-learn==0.5.3 altair==5.0.1 numba==0.57.1 numpy==1.24.4


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [512]:
import pandas as pd
import os
import cohere
import umap
import altair as alt
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

## Carregando e tratando o dataset de filmes

### Importando os dados

In [513]:
dataset = pd.read_csv('data/movies_metadata.csv', usecols=['id','original_language','original_title','overview','release_date','adult'])
dataset.head(2)

,adult,id,original_language,original_title,overview,release_date
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15


### Selecionando apenas os filmes brasileiros

In [514]:
filmes_brasileiros = dataset[dataset['original_language'] == 'pt']

In [515]:
filmes_brasileiros.info()

<class 'pandas.core.frame.DataFrame'>
Index: 316 entries, 738 to 45168
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   adult              316 non-null    object
 1   id                 316 non-null    object
 2   original_language  316 non-null    object
 3   original_title     316 non-null    object
 4   overview           295 non-null    object
 5   release_date       315 non-null    object
dtypes: object(6)
memory usage: 17.3+ KB


Em nossa análise utilizaremos os *campos original_title* e *overview*. Para isso precisamos retirar os registros nulos destes dois campos.

### Retirando os campos nulos

In [516]:
filmes_brasileiros = filmes_brasileiros.dropna(subset='overview')

In [517]:
filmes_brasileiros.head(2)

,adult,id,original_language,original_title,overview,release_date
738,False,255546,pt,Carmen Miranda: Bananas Is My Business,A biography of the Portuguese-Brazilian singer...,1995-04-13
1659,False,12538,pt,Senseless,A student gets his senses enhanced by an exper...,1998-02-20


In [518]:
print(f"Existem {filmes_brasileiros.shape[0]} filmes brasileiros neste dataset aptos para nossa investigação")

Existem 295 filmes brasileiros neste dataset aptos para nossa investigação


## Extraindo o embedding dos títulos dos filmes

Para o mapeamento dos embeddings utilizaremos uma biblioteca Python da ***Cohere***. 

Neste processo temos três tipos de modelos para embedding:

- embed-english-v2.0 (padrão) 4096 dimensões
- embed-english-light-v2.0 1024 dimensões
- embed-multilingual-v2.0 768 dimensões

Nesta parte vamos utilizar apenas os títulos originais em português com um modelo Multilingual (embed-multilingual-v2.0)

Fonte: https://docs.cohere.com/reference/embed

In [519]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [520]:
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [521]:
emb = co.embed(texts=list(filmes_brasileiros.original_title),
               model='embed-multilingual-v2.0').embeddings

In [522]:
print(f"O embedding da Cohere Multilingual é um vetor de {len(emb[0])} dimensões")

O embedding da Cohere Multilingual é um vetor de 768 dimensões


### Diminuindo a dimensão

Nosso objetivo neste primeiro estudo é ter uma noção visual de como os títulos de filmes se relacionam uns com os outros através de um plano cartesiano, o problema é que para nós humanos é muito difícil imaginar um plano com 768 dimensões.

Sendo assim precisamos diminuir as 768 dimensões do ***embedding*** para três ou duas dimensões. No caso vamos diminuí-la para duas.

A biblioteca ***umap*** reduzirá as 768 dimensões em duas

In [523]:
reducer = umap.UMAP(n_neighbors=2)
umap_embeds_2d = reducer.fit_transform(emb)
umap_embeds_2d[:2]

array([[10.920109,  9.097715],
       [11.158794, 14.357875]], dtype=float32)

In [524]:
df_embeds_d2 = filmes_brasileiros.original_title.to_frame().copy()
df_embeds_d2['x'] = umap_embeds_2d[:,0]
df_embeds_d2['y'] = umap_embeds_2d[:,1]
df_embeds_d2.head()

,original_title,x,y
738,Carmen Miranda: Bananas Is My Business,10.920109,9.097715
1659,Senseless,11.158794,14.357875
1670,"O Que é Isso, Companheiro?",-4.761305,2.486226
1806,Viagem ao Princípio do Mundo,-7.816003,-7.074599
2244,Central do Brasil,7.406845,2.986557


No DataFrame acima temos a nossa dimensionalidade reduzida para cada título de filme, representada pelas colunas ***x*** e ***y***

### Plotando os filmes

Agora chegou a hora de colocar todos os filmes no plano cartesiano de duas dimensões e olhar como eles estão relacionados. 

Este relacioamento é dado exatamente pela proximidade entre eles no plano

In [525]:
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

chart = alt.Chart(df_embeds_d2).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['original_title']
).properties(
    width=700,
    height=400
)

text = chart.mark_text(
    align='left',
    baseline='middle',
).encode(
    text='original_title'
)

chart_with_labels = (chart + text)

## Plota apenas os pontos que representam os filmes no plano
chart.interactive()

## Plota os pontos e os títulos no plano
#chart_with_labels.interactive()

alt.Chart(...)

Para aqueles que não conseguirem rodar o notebook insiro abaixo a imagem do mapeamento gerado acima

![Alt text](data/visualization-1.svg)

O mapeamento nos mostra como os filmes (pontos azuis) se relacionam a partir de seus títulos. 

Podemos notar alguns aglomerados que se formam exatamente por sua maior proximidade. Vamos olhar alguns deles.

![Alt text](data/visualization-2.svg)

Um agrupamento de títulos em inglês.

![Alt text](data/visualization-3.svg)

Agrupamento interessante de nomes indígenas e Amazônia

![Alt text](data/visualization-4.svg)

Este agrupamento de filmes não parecem ter muito a ver.

## Extraindo o embedding da descrição dos filmes

Vamos observar agora como um mapeamento utilizando o resumo do filme, ao invés do título, pode modificar as relações de proximidade.

Diferente do título original, ***overview*** neste banco de dados está em inglês. Por isso vamos utilizar o modelo embed-english-light-v2.0

In [526]:
filmes_brasileiros.overview.to_frame()

,overview
738,A biography of the Portuguese-Brazilian singer...
1659,A student gets his senses enhanced by an exper...
1670,"Fernando, a journalist, and his friend César j..."
1806,Manoel is aging film director who travels with...
2244,"An emotive journey of a former school teacher,..."
...,...
44541,"For forty years, the public functionary and fa..."
44591,It's a love story between a fashion blogger an...
44941,Pierre is seventeen and in the middle of puber...
44977,A group of honest police is sent to a country ...


In [527]:
emb_overview = co.embed(texts=list(filmes_brasileiros.overview),
               model='embed-english-light-v2.0').embeddings

In [528]:
emb_overview
print(f"O embedding da Cohere English Light é um vetor de {len(emb_overview[0])} dimensões")

O embedding da Cohere English Light é um vetor de 1024 dimensões


### Diminuindo a dimensão

In [529]:
reducer_overview = umap.UMAP(n_neighbors=2)
umap_embeds_overview = reducer_overview.fit_transform(emb_overview)
umap_embeds_overview[:2]

/usr/local/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


array([[  5.6369987,  -5.575538 ],
       [  9.817616 , -11.427569 ]], dtype=float32)

In [530]:
df_embeds_overview = filmes_brasileiros.original_title.to_frame().copy()
df_embeds_overview['x'] = umap_embeds_overview[:,0]
df_embeds_overview['y'] = umap_embeds_overview[:,1]
df_embeds_overview.head()

,original_title,x,y
738,Carmen Miranda: Bananas Is My Business,5.636999,-5.575538
1659,Senseless,9.817616,-11.427569
1670,"O Que é Isso, Companheiro?",3.550342,5.383621
1806,Viagem ao Princípio do Mundo,7.124527,7.471398
2244,Central do Brasil,13.843379,5.666356


### Plotando os filmes

In [531]:
chart = alt.Chart(df_embeds_overview).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['original_title']
).properties(
    width=700,
    height=400
)

text = chart.mark_text(
    align='left',
    baseline='middle',
).encode(
    text='original_title'
)

chart_with_labels = (chart + text)
#chart_with_labels.interactive()
chart.interactive()

alt.Chart(...)

Para aqueles que não conseguirem rodar o notebook insiro abaixo a imagem do mapeamento gerado acima

![Visão completa do mapeamento por overview](data/visualization-5.svg)

Visualmente podemos notar que as distribuições mudaram e novos aglomerados surgiram.

![Curumim](data/visualization-6.svg)

Aqui já notamos que o filme Curumim não está mais próximo a 'Amazônia' mas sim a filmes ligados a tráfico de drogas e violência.

In [532]:
filmes_brasileiros[filmes_brasileiros.original_title == 'Curumim'].loc[42268]['overview']

'Marco ‘Curumim’ Archer’s life changed abruptly when police at Jakarta airport seize 13.5 kilogrammes of cocaine hidden in his hang-glider. At first he manages to escape, hiding out in Indonesia for sixteen days before being arrested and sentenced to death. Eleven years later, on 17 January 2015, he was executed for drug dealing.'

Pela descrição do overview percebemos ele está no agrupamento certo de filmes.

![Curumim](data/visualization-7.svg)

o filme 'Amazonia' ganhou novos parentes mas 'Macunaíma' permanece próximo pois ambos estão ligados pela questão indígena.

![Curumim](data/visualization-8.svg)

'Liar, Liar, Vampire' já não aparece agrupado em títulos em inglês.

Vamos dar uma olhada no overview dos três filmes.

In [533]:
filmes_brasileiros[filmes_brasileiros.original_title == 'Liar, Liar, Vampire'].loc[42237]['overview']

"When an ordinary boy Davis, suddenly becomes famous at school as people start to believe he's actually a vampire, vampire expert Cameron helps him act like a real vampire."

In [534]:
filmes_brasileiros[filmes_brasileiros.original_title == 'Eu Não Quero Voltar Sozinho'].loc[21062]['overview']

"The arrival of a new student in school changes Leonardo's life. This 15 year-old blind teenager has to deal with the jealousy of his friend Giovana while figuring out the new feelings he's having towards his new friend, Gabriel."

In [535]:
filmes_brasileiros[filmes_brasileiros.original_title == 'Hoje Eu Quero Voltar Sozinho'].loc[24886]['overview']

'Leonardo is a blind teenager dealing with an overprotective mother while trying to live a more independent life. To the disappointment of his best friend, Giovana, he plans to go on an exchange program abroad. When Gabriel, a new student in town, arrives at their classroom, new feelings blossom in Leonardo making him question his plans.'

Pela descrição dos três filmes notamos que a temática está ligada a estudantes, escola e jovem, portanto faz sentido estarem agrupados.

### Conclusão

Os embeddings traduzem a linguagem humana para uma linguagem vetorial e com isso conseguimos fazer com que a máquina relacione palavras e textos entre sí. É como se a máquina entendesse o significado da lingua humana utilizando as relações entre palavras e frases.

# <span style="color: #6850e5;">Busca por similaridade de filmes em inglês</span>

Buscas são a chave de inúmeros negócios que estão imersos em grande quantidade de informação.

Vamos utilizar os ***embeddings*** estudados na sessão anterior para montar esta busca.

A partir do nosso catálogo de filmes e ***embeddings*** seguimos na construção de uma busca por similaridade utilizando desta vez apenas os filmes em inglês.

O desenvolvimento do buscador seguirá as seguintes etapas:

1) Filtrar e tratar os dados originais do nosso dataset
2) Preparar o dataset para o mapeamento de embeddings
3) Extrair os embeddings dos filmes
4) Montar a busca utilizando proximidade vetorial
5) Montar uma saída para a resposta baseada em Generative AI

### Instalando e importando bibliotecas

In [536]:
!pip install -q annoy==1.17.3


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [537]:
import numpy as np
from annoy import AnnoyIndex

### Selecionando apenas os filmes em inglês

In [538]:
filmes_en = dataset[dataset['original_language'] == 'en']

In [539]:
filmes_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32269 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   adult              32269 non-null  object
 1   id                 32269 non-null  object
 2   original_language  32269 non-null  object
 3   original_title     32269 non-null  object
 4   overview           32200 non-null  object
 5   release_date       32202 non-null  object
dtypes: object(6)
memory usage: 1.7+ MB


### Retirando os campos nulos

Utilizaremos os campos *origial_title* e *overview*, sendo assim precisamos retirar os registros nulos

In [540]:
filmes_en = filmes_en.dropna(subset=['original_title','overview'])

In [541]:
filmes_en.head(2)

,adult,id,original_language,original_title,overview,release_date
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15


In [542]:
print(f"Existem {filmes_en.shape[0]} filmes no idioma inglês neste dataset aptos para nossa investigação")

Existem 32200 filmes no idioma inglês neste dataset aptos para nossa investigação


Por questões de velocidade e gasto da API vamos reduzir esta quantidade de filmes selecionando aleatoriamente 300 filmes

In [543]:
filmes_en = filmes_en.sample(n=300, random_state=2758)
filmes_en.shape

(300, 6)

### Preparando o conteúdo para o mapeamento

Para que nossa busca fique mais robusta vamos juntar o *original_title* com *overview* e utilizar este novo campo como base para o mapeamento de embeddings.

In [544]:
filmes_en['title_and_overview'] = filmes_en['original_title'] + ' ' + filmes_en['overview']
filmes_en.head(2)

,adult,id,original_language,original_title,overview,release_date,title_and_overview
25339,False,55322,en,Grant Morrison: Talking with Gods,"Philosopher, pop-icon, shaman and story teller...",2010-10-26,Grant Morrison: Talking with Gods Philosopher...
8225,False,48678,en,Falling Angels,The wickedly funny story of three sisters' com...,2003-09-08,Falling Angels The wickedly funny story of thr...


Vamos resetar o índice do DataFrame para facilitar o mapeamento e a posterior busca

In [545]:
filmes_en.reset_index(inplace=True)
filmes_en.drop(['index'], axis=1, inplace=True)

In [546]:
filmes_en.head(2)

,adult,id,original_language,original_title,overview,release_date,title_and_overview
0,False,55322,en,Grant Morrison: Talking with Gods,"Philosopher, pop-icon, shaman and story teller...",2010-10-26,Grant Morrison: Talking with Gods Philosopher...
1,False,48678,en,Falling Angels,The wickedly funny story of three sisters' com...,2003-09-08,Falling Angels The wickedly funny story of thr...


### Mapeando os embeddings dos filmes

Agora vamos mapear estes 300 filmes utilizando os embeddings da ***Cohere***

In [547]:
filmes_en.title_and_overview.tolist()[:2]

['Grant Morrison:  Talking with Gods Philosopher, pop-icon, shaman and story teller - as one of the most influential comic book writers in the market today, Grant Morrison is all of these things. His explosive and often controversial 30 year career has made him a household name for comic fans, and he shows no signs of slowing down. Morrison\'s career is now taking a new turn, as he explores media beyond comic books. With film and television projects on the horizon, more and more people are asking "Who is Grant Morrison?" Grant Morrison: Talking with Gods is a feature length documentary that takes an in depth look at the life, career and mind of the man behind such pivotal titles as Batman RIP, The Invisibles, All Star Superman, The New X-Men, and many more. Featuring candid interviews with Morrison and his most important collaborators, Talking with Gods reveals an intelligent and thought provoking side of comic books that is often overlooked in Hollywood adaptations.',
 'Falling Angels

In [548]:
embedding_en = co.embed(
    texts=filmes_en.title_and_overview.tolist(),
).embeddings

In [549]:
print(f"Cada embedding de um overview possui {len(embedding_en[0])} dimensões")

Cada embedding de um overview possui 4096 dimensões


In [550]:
embeds = np.array(embedding_en)
embeds[:2]

array([[ 3.9042969 , -1.4189453 ,  1.359375  , ...,  0.32983398,
        -2.6445312 ,  0.59277344],
       [ 2.09375   , -2.5820312 ,  0.5078125 , ..., -0.95751953,
         0.12017822, -0.1541748 ]])

Agora que os embeddings estão no formato correto, vamos criar um buscador vetorial.

### Criando um buscador vetorial de dados

Para isso utilizaremos a biblioteca ***Annoy*** desenvolvida pelo *Spotify*

Referência: https://www.youtube.com/watch?v=DRbjpuqOsjk

In [551]:
# Cria um índice de busca, informando a dimensão do embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')

In [552]:
print(f'Aqui nós instanciamos o Annoy informando a ele que nossos embeddings possuem {embeds.shape[1]} dimensões')

Aqui nós instanciamos o Annoy informando a ele que nossos embeddings possuem 4096 dimensões


Agora vamos adicionar nesta instância criada acima cada um dos 300 overviews convertidos para embeddings

In [553]:
for i in range(len(embeds)):
    #print(i, embeds[i])
    search_index.add_item(i, embeds[i])

Vamos finalizar o mapeamento instruindo o ***Annoy*** a mapear nossos 300 overviews utilizando uma árvore de tamanho 10. 

In [554]:
search_index.build(10)

## Para este projeto é opcional salvar um arquivo com o mapeamento
#search_index.save('filmes_en.ann')

True

Nosso *search_index* está pronto com todo o mapeamento.

### Implementando a busca

A busca precisa cumprir alguns passos:

1) Converter a pergunta para embeddings
2) Pegar este embedding da pergunta e buscar em nosso catálogo de 300 filmes o embedding que se aproxima mais
3) Após encontar os índices dos mais próximos, utilizá-los para encontar os títulos e overviews originais

In [555]:
def busca(pergunta):
    # Faz o embedding da pergunta
    embed_pergunta = co.embed(texts=[pergunta]).embeddings
    
    # Compara a pergunta com o catálogo de 300 filmes e retorna os 10 vizinhos mais próximos da pergunta
    similar_item_ids = search_index.get_nns_by_vector(embed_pergunta[0], 10, include_distances=True)
    
    # Após obter os 10 IDs selecionamos o mais relevante, ou seja, o primeiro
    resultado_da_busca = filmes_en.loc[similar_item_ids[0]]
    
    return (embed_pergunta[0], resultado_da_busca[['original_title','overview','release_date']])

In [556]:
pergunta = "Recommend me a movie to watch with my wife"
emb_pergunta, results = busca(pergunta=pergunta)

In [557]:
results

,original_title,overview,release_date
211,The Perfect Husband,A woman discovers her new husband wants to kee...,2004-02-12
275,"Lovely, Still",A holiday fable that tells the story of an eld...,2008-01-01
45,Blind Date,"Affected by tragedy, a married couple decide t...",2007-05-16
31,Let It Be Me,"Right after getting engaged, a man starts taki...",1995-11-17
299,Dinner for One,A very old woman wants to have dinner with her...,1963-06-08
197,The Face of Love,A widow falls for a guy who bears a striking r...,2013-10-25
199,Ramona,Half-Indian girl brought up in a wealthy house...,1936-09-25
36,To Sleep with Anger,An enigmatic drifter from the South comes to v...,1990-09-11
21,A Decade Under the Influence,Documentary on the effects of 1970s filmmaking.,2003-04-25
60,Dagmars Heta Trosor,"Known to her clients as Dagmar, she's a classy...",1971-10-01


Finalmente temos o DataFrame com os 10 filmes que mais se aproximam da pergunta feita.

### Formatando a saída da busca

Vamos deixar o resultado da busca um pouco mais claro.

In [558]:
title = results.iloc[0]['original_title']
overview = results.iloc[0]['overview']
print(f'A busca mais relevante para a pergunta (em inglês):\n\n"{pergunta}"\n\né o filme:\n"{title}"\n\nResumo (em inglês):\n"{overview}"')

A busca mais relevante para a pergunta (em inglês):

"Recommend me a movie to watch with my wife"

é o filme:
"The Perfect Husband"

Resumo (em inglês):
"A woman discovers her new husband wants to keep her all to himself."


### Plotando a busca e as respostas

Vamos olhar graficamente como a pergunta e a resposta anterior estão relacionadas, ou seja, sua proximidade.

#### Reduzindo a dimensão da pergunta e dos filmes

In [559]:
na = np.append(embeds, np.array([emb_pergunta]), axis=0)
reducer = umap.UMAP(n_neighbors=2)
en_embeds_2D = reducer.fit_transform(na)
en_embeds_2D[:2]

/usr/local/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


array([[15.054978  , 12.257607  ],
       [10.04974   , -0.41221595]], dtype=float32)

In [560]:
df_en_embeds_d2 = filmes_en.original_title.to_frame().copy()
df2 = pd.DataFrame([[pergunta]], columns=['original_title'])
df_en_embeds_d2 = pd.concat([df_en_embeds_d2,df2], ignore_index=True)

df_en_embeds_d2['x'] = en_embeds_2D[:,0]
df_en_embeds_d2['y'] = en_embeds_2D[:,1]

df_en_embeds_d2.tail()

,original_title,x,y
296,The Halliday Brand,11.775882,6.330390
297,Holiday Inn,18.077997,2.309694
298,Drillbit Taylor,17.554157,7.328094
299,Dinner for One,11.395221,10.832385
300,Recommend me a movie to watch with my wife,12.148176,6.554398


#### Plotando filmes + pergunta

In [561]:
chart = alt.Chart(df_en_embeds_d2).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['original_title']
).properties(
    width=700,
    height=400
)

text = chart.mark_text(
    align='left',
    baseline='middle',
).encode(
    text='original_title'
)

chart_with_labels = (chart + text)
chart_with_labels.interactive()

alt.LayerChart(...)

![](data/visualization-9.svg)

Acima está a imagem de um zoom exibindo a posição da pergunta "Recommend me a movie to watch with my wife" e os filmes mais próximos, no caso o mais próximo é o "The Perfect Husband" que é nossa resposta final.

### Resposta gerada por Generative AI

Podemos melhorar a resposta de modo a parecer uma pessoa incluindo AI Generativa.

Referência: https://docs.cohere.com/reference/generate

A resposta precisa cumprir alguns passos:

1) Receber a pergunta e o resultado da busca
2) Criar um prompt inserindo estes dados e explicando como responder a partir dos dados fornecidos
3) Enviar o prompt para a API da ***Cohere*** com as configurações necessárias para uma boa resposta
4) Retornar o resultado

***OBS***: A resposta da Generative API pode variar a cada vez que for rodada pois é uma característica das AIs generativas.

In [562]:
def pergunte_sobre_filmes(pergunta, resultado, num_generations=1):
    # Resultado mais relevante.
    titulo = resultado.iloc[0]['original_title']
    resumo = resultado.iloc[0]['overview']

    # Prepara o prompt
    prompt = f"""
    User's input: {pergunta}.
    Title and overview of movie than matches the user's input: {titulo} - {resumo}.
    
    Return an answer up to 60 words to user based on text above.
    """

    # Chama a API Generative AI passando o prompt as configurações necessárias para a resposta
    predicao = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return predicao.generations

In [563]:
resposta = pergunte_sobre_filmes(pergunta="Recommend me a movie to watch with my wife", resultado=results)
resposta[0]

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


,prompt,text
0,User's input: Recommend me a movie to watch with my wife. Title and overview of movie than matches the user's input: The Perfect Husband - A woman discovers her new husband wants to keep her all to himself.. Return an answer up to 60 words to user based on text above.,"A thriller about a woman who has to outsmart her new husband to stay alive! It's a great movie for a date night, with twists and turns that will keep you on the edge of your seat. This film, directed by Michael Sucsy, is a great date night movie that will keep you entertained and thrilled!"


In [564]:
str(resposta[0])

"A thriller about a woman who has to outsmart her new husband to stay alive!  It's a great movie for a date night, with twists and turns that will keep you on the edge of your seat. \nThis film, directed by Michael Sucsy, is a great date night movie that will keep you entertained and thrilled!  \n"

## <span style="color: #6850e5;">Conclusão</span>

Com a utilização de ***embeddings*** de uma LLM e ***Generative AI*** foi possivel desenvolver um sistema de recomendação de filmes baseado em um dataset.

Porém antes disso vimos como os ***embeddings*** nos ajudam a mapear os relacionamentos entre textos baseados em seus contextos. É a partir da proximidade destes relacionamentos que conseguimos relacionar também uma pergunta com a melhor resposta, que seria aquela mais próxima vetorialmente.

Com o avanço das tecnologias de AI e a disponibilização de APIs para interagir com estas AIs temos uma enorme gama de coisas que podemos fazer. As ferramentas disponibilizadas mais as técnicas de manipulação de dados e programação podem nos proporcionar aplicações impensáveis há alguns anos atrás.